---
title: pyvista + ipywidgets volume rendering
author: Georgios Varnavides
date: 2023/07/30
---

This is an example widget illustrating how to use pyvista and ipywidgets together for 3D volume rendering.

In [56]:
#| label: app:pyvista_interactive_ipywidgets
# Widget combining pyvista + ipywidgets for volume rendering.

from IPython.display import display
import matplotlib.pyplot as plt
import pyvista as pv
import vtk
from ipywidgets import Checkbox, IntSlider, Layout, HBox, VBox, Label, Dropdown
import numpy as np
import h5py

# Data
with h5py.File("data/CNT_overlap_tomo_missing.h5","r") as f:
    values = f['reconstruction'][:]

nx, ny, nz = values.shape

grid = pv.ImageData(
    dimensions=[nx+1,ny+1,nz+1],
    origin=(0,0,0),
    spacing=(1,1,1),
)
grid.cell_data["values"] = values.flatten(order="F")

pl = pv.Plotter()

vol = pl.add_volume(
    grid,
    name='volume',
    opacity="sigmoid",
    cmap='magma',
    show_scalar_bar=False,
)

grid_slice_x = grid.slice(
    normal='x',
    origin=(nx//2,ny//2,nz//2),
)

grid_slice_y = grid.slice(
    normal='y',
    origin=(nx//2,ny//2,nz//2),
)

grid_slice_z = grid.slice(
    normal='z',
    origin=(nx//2,ny//2,nz//2),
)

volume_clip_plane = pl.add_volume_clip_plane(
    vol,
    normal='x',
    origin=(0,ny//2,nz//2),
)
volume_clip_plane.Off()

slice_x = pl.add_mesh(
    grid_slice_x,
    show_scalar_bar=False,
    cmap='magma',
    opacity=0.85,
    name="slice-x"
)

slice_y = pl.add_mesh(
    grid_slice_y,
    show_scalar_bar=False,
    cmap='magma',
    opacity=0.85,
    name="slice-y"
)

slice_z = pl.add_mesh(
    grid_slice_z,
    show_scalar_bar=False,
    cmap='magma',
    opacity=0.85,
    name="slice-z"
)

slices = [slice_x,slice_y,slice_z]

for slice in slices:
    slice.SetVisibility(False)

pl.camera.zoom(0.75)

viewer = pl.show(
    return_viewer = True,
    interactive_update=True,
    window_size=(400,400),
)

def update_x_slice(change):
    offset = nx-change['new']
    cy = ny - cy_slider.value
    cz = nz - cz_slider.value
    
    pl.remove_actor(slices[0])

    grid_slice_x = grid.slice(
        normal='x',
        origin=(offset,cy,cz),
    )
    
    slices[0] = pl.add_mesh(
        grid_slice_x,
        show_scalar_bar=False,
        cmap='magma',
        opacity=0.85,
        name="slice-x"
    )
    pl.update()
    return None

def update_y_slice(change):
    offset = ny-change['new']
    cx = nx - cx_slider.value
    cz = nz - cz_slider.value
    
    pl.remove_actor(slices[1])

    grid_slice_y = grid.slice(
        normal='y',
        origin=(cx,offset,cz),
    )
    
    slices[1] = pl.add_mesh(
        grid_slice_y,
        show_scalar_bar=False,
        cmap='magma',
        opacity=0.85,
        name="slice-y"
    )
    pl.update()
    return None

def update_z_slice(change):
    offset = nz-change['new']
    cx = nx - cx_slider.value
    cy = ny - cy_slider.value
    
    pl.remove_actor(slices[2])

    grid_slice_z = grid.slice(
        normal='z',
        origin=(cx,cy,offset),
    )
    
    slices[2] = pl.add_mesh(
        grid_slice_z,
        show_scalar_bar=False,
        cmap='magma',
        opacity=0.85,
        name="slice-z"
    )
    pl.update()
    return None

def toggle_x_slice(change):
    visible = change['new']
    slices[0].SetVisibility(visible)
    pl.update()
    return None

def toggle_y_slice(change):
    visible = change['new']
    slices[1].SetVisibility(visible)
    pl.update()
    return None

def toggle_z_slice(change):
    visible = change['new']
    slices[2].SetVisibility(visible)
    pl.update()
    return None

def toggle_clip_plane(change):
    visible = change['new']
    if visible:
        volume_clip_plane.On()
    else:
        volume_clip_plane.Off()
    pl.update()
    return None

def toggle_vol(change):
    visible = change['new']
    vol.SetVisibility(visible)
    pl.update()
    return None

# Widgets 
sequential_cmaps = [
    'viridis', 'plasma', 'inferno', 'magma', 'cividis','turbo',
    'Greys', 'Purples', 'Blues', 'Greens', 'Oranges', 'Reds',
    'YlOrBr', 'YlOrRd', 'OrRd', 'PuRd', 'RdPu', 'BuPu',
    'GnBu', 'PuBu', 'YlGnBu', 'PuBuGn', 'BuGn', 'YlGn'
]

def update_colormap(change):
    cmap_string = change['new']
    cf = vtk.vtkColorTransferFunction()
    cmap = pv.LookupTable(cmap_string)
    
    for ii in range(cmap.n_values):
        cf.AddRGBPoint(ii, *cmap(ii/(cmap.n_values-1))[:-1])
    
    vol.prop.SetColor(0,cf)
    pl.update()
    return None

cmap_widget =Dropdown(options=sequential_cmaps,value='magma',description="Colormap",indent=False,layout=Layout(width='175px'))
cmap_widget.observe(update_colormap,names='value')

checkbox_layout = Layout(width='175px')

vol_checkbox = Checkbox(
    value=True,
    description='Show Vol',
    indent=True,
    layout=checkbox_layout,
)
vol_checkbox.observe(toggle_vol,names='value')

clip_checkbox = Checkbox(
    value=False,
    description='Clip Plane',
    indent=True,
    layout=checkbox_layout,
)
clip_checkbox.observe(toggle_clip_plane,names='value')

cx_checkbox = Checkbox(
    value=False,
    description='x axis slice',
    indent=True,
    layout=checkbox_layout,
)
cx_checkbox.observe(toggle_x_slice,names='value')

cy_checkbox = Checkbox(
    value=False,
    description='y axis slice',
    indent=True,
    layout=checkbox_layout,
)
cy_checkbox.observe(toggle_y_slice,names='value')

cz_checkbox = Checkbox(
    value=False,
    description='z axis slice',
    indent=True,
    layout=checkbox_layout,
)
cz_checkbox.observe(toggle_z_slice,names='value')

slider_layout = Layout(width='175px')
cx_slider = IntSlider(
    value=nx//2,
    min=0,
    max=nx-1,
    description='x index',
    continuous_update=False,
    layout=slider_layout,
    readout=False,
)
cx_slider.observe(update_x_slice,names='value')

cy_slider = IntSlider(
    value=ny//2,
    min=0,
    max=ny-1,
    description='y index',
    continuous_update=False,
    layout=slider_layout,
    readout=False,
)
cy_slider.observe(update_y_slice,names='value')

cz_slider = IntSlider(
    value=nz//2,
    min=0,
    max=nz-1,
    description='z index',
    continuous_update=False,
    layout=slider_layout,
    readout=False,
)
cz_slider.observe(update_z_slice,names='value')

controls_layout = Layout(
    display='flex',
    flex_flow='column',
    align_items='center',
    width='200px'
)

controls = VBox(
    [
        Label("Volume Controls"),
        cmap_widget,
        vol_checkbox,
        clip_checkbox,
        Label("Slice Controls"),
        cx_checkbox,
        cy_checkbox,
        cz_checkbox,
        Label("Indices Controls"),
        cx_slider,
        cy_slider,
        cz_slider
    ],
    layout=controls_layout
)

visualization_layout = Layout(
    display='flex',
    flex_flow='row',
    align_items='center',
    width='600px'
)

display(
    HBox([
        viewer,
        controls
    ],
        layout=visualization_layout
        )
)